# Single step agent

1. Setup prompt that asks an LLM to use provided tools or use tool responses to generate answers
2. Setup the tools, decriptions
3. **Single Step Agent**:
   - Ask LLM to pick the tools to run
   - Runs the tools
   - Send the tools responses to LLM to generate final answer
4. Test scenarios

Note:
* Code uses the OpenAI GPT 3.5 turbo model

## Setup LLM
Same LLM will be used for planning & answering.

In [1]:
from dotenv import load_dotenv
import sys
import json

from langchain.prompts import PromptTemplate

# Load the file that contains the API keys - OPENAI_API_KEY
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

# setting path
sys.path.append('../')

from utils.create_llm import create_gpt_llm, create_anthropic_llm, create_ai21_llm, create_cohere_llm

# Try with GPT
llm = create_gpt_llm()




NameError: name 'ChatOpenAI' is not defined

## 1. Setup the prompt

In [2]:
template = """
You are a helpful assistant who can answer questions on a variety of topics. 
Do not use your own internal knowledge or information to answer the question. 
Think step by step and use only the following available tools.

Tools:
{tools}

There is no need for preamble, just give the response in ONE of the following valid JSON formats:

option 1:
use this if you need tools to be run to get the information
{{"actions": [{{ "action" : tool name, "arguments" : dictionary of argument values}}}}]

option 2:
use this if you can use the action responses to answer the question
{{"answer": "your response to the question", "explanation": "provide your explanation here"}}

Here is the question: {question}

action_responses: {action_responses}

response:

"""

prompt = PromptTemplate(
    template = template,
    input_variables = ['tools', 'question', 'action_responses']
)

## 2. Setup tools
* Dummy tool functions for stock price & city weather
* Tool map = dictionary of functions with function name as the key


In [3]:
## Tool 1 for stocks
def  company_stock_price(stock_symbol: str) -> float:
    if stock_symbol.upper()=='APPL':
        return  {"price": 192.32}
    elif stock_symbol.upper()=='MSFT':
        return  {"price": 415.60}
    elif stock_symbol.upper()=='AMZN':
        return  {"price": 183.60}
    else:
        return {"price": "unknown"}
    pass

stock_tool_description = {
    "name" : "company_stock_price",
    "description": "This tool returns the last known stock price for a company",
    "arguments": [
        {"stock_symbol" : "stock exchange symbol for the company"}
    ],
    "response": "last known stock price"
}

## Tool 2 for city weather
def city_weather(city: str) -> int:
    if city.lower() == "new york":
        return {"temperature": 68, "forecast": "rain"}
    elif city.lower() == "paris":
        return {"temperature": 73, "forecast": "sunny"}
    elif city.lower() == "london":
        return {"temperature": 82, "forecast": "cloudy"}
    else:
        return {"temperature": "unknown"}
        
city_weather_tool_description = {
    "name" : "city_weather",
    "description": "This tool returns the current temperature and forecast for the given city",
    "arguments": [
        {"city" : "name of the city"}
    ],
    "response": "current temperature & forecast"
}

tools = [stock_tool_description, city_weather_tool_description]
tools_map = {
    'company_stock_price': company_stock_price,
    'city_weather' : city_weather
}

## 3. Agent code

In [4]:
# Utility function to process the actions receieved from the LLM
def   process_response_actions(response):
    action_responses = []
    if len(response["actions"]) == 0:
        print('question cannot be answered as there is no tool to use !!!')
        exit
    else:
        for action in response["actions"]:
            action_function = tools_map[action["action"]]
            action_invoke_result = action_function(**action["arguments"])
            action["response"] = action_invoke_result
            action_responses.append(action)
            
    return action_responses

In [5]:
# Utility function for answering the question
def  process_query(question):
    # Setup the prompt. Since no tool has been invoked set action_response as blank
    query = prompt.format(tools=tools, question=question,action_responses="")

    # Invoke LLM to get the tools to be run
    response = llm.invoke(query)

    # print the response
    print("Step-1:", response)

    # Convert response to JSON object
    response_json = json.loads(response)

    # LLM may respond with an answer 
    # It may happen if LLM determines that no tool is available for responding to the question
    action_responses=[]
    if "answer" in response_json:
        return {"answer" : response_json["answer"]}
    elif "actions" in response_json:
        action_responses = process_response_actions(response_json)

    print("Agent tool invocation responses :", action_responses)
    
    # Now send the action responses to LLM for generating the answer
    query = prompt.format(tools=tools, question=question,action_responses=action_responses)
    response = llm.invoke(query)

    # print the response
    print("Step-2:", response)

    # Convert response to JSON object
    response_json = json.loads(response)

    # Extract the answer from the response    
    if "answer" in response_json:
        return response_json["answer"]
    else:
        return ("Something unexpected has happened !!!")

## Test

### 1. Simple test that uses 1 tool

In [6]:
question = "Which of these cities is hotter, Paris or London"
question = "I am visting paris, should i carry an umbrella?"

response = process_query(question)

print("Final response::",response)

Step-1: {"actions": [{"action": "city_weather", "arguments": {"city": "paris"}}]}
Agent tool invocation responses : [{'action': 'city_weather', 'arguments': {'city': 'paris'}, 'response': {'temperature': 73, 'forecast': 'sunny'}}]
Step-2: {"answer": "No, you should not carry an umbrella.", "explanation": "The current forecast for Paris is sunny and the temperature is 73 degrees, so an umbrella is not necessary."}
Final response:: No, you should not carry an umbrella.


### 2. Test for a scenario when no appropriate tool is available

In [7]:
question = "I have only $200, can I afford to live in Paris for 3 nights?"

response = process_query(question)

print("Final response:",response)

Step-1: {"actions": [
    {"action": "city_weather", "arguments": {"city": "Paris"}},
    {"action": "company_stock_price", "arguments": {"stock_symbol": "EUR"}}
]}
Agent tool invocation responses : [{'action': 'city_weather', 'arguments': {'city': 'Paris'}, 'response': {'temperature': 73, 'forecast': 'sunny'}}, {'action': 'company_stock_price', 'arguments': {'stock_symbol': 'EUR'}, 'response': {'price': 'unknown'}}]
Step-2: {"answer": "No", "explanation": "The current temperature in Paris is 73 degrees and the forecast is sunny. However, the stock price for EUR (Euro) is unknown, so we cannot determine if you can afford to live in Paris for 3 nights with only $200."}
Final response: No


### 3. A complex scenario requiring use of more than one tool

In [9]:
question = """ 
I am interested in buying one of the stocks (AAPL, MSFT, AMZN). 
I will make my decision based on the weather forecast in the city I am in and the price of the stock.
If the weather is sunny then I will chose a stock that has a lower price.
If the weather is raining then I will chose a stock that has a higher price.
If the wwather is cloudy then I will not buy any stock.

Today I am in London, which stock should I buy?

"""

response = process_query(question)

print("Final response:",response)

Step-1: {"actions": [
    {"action": "city_weather", "arguments": {"city": "London"}},
    {"action": "company_stock_price", "arguments": {"stock_symbol": "AAPL"}},
    {"action": "company_stock_price", "arguments": {"stock_symbol": "MSFT"}},
    {"action": "company_stock_price", "arguments": {"stock_symbol": "AMZN"}}
]}
Agent tool invocation responses : [{'action': 'city_weather', 'arguments': {'city': 'London'}, 'response': {'temperature': 82, 'forecast': 'cloudy'}}, {'action': 'company_stock_price', 'arguments': {'stock_symbol': 'AAPL'}, 'response': {'price': 'unknown'}}, {'action': 'company_stock_price', 'arguments': {'stock_symbol': 'MSFT'}, 'response': {'price': 415.6}}, {'action': 'company_stock_price', 'arguments': {'stock_symbol': 'AMZN'}, 'response': {'price': 183.6}}]
Step-2: {"answer": "I would not recommend buying any of the stocks (AAPL, MSFT, AMZN) based on the current weather forecast in London, which is cloudy. It would be best to wait for a different weather condition